### 한겨레일보 데이터크롤러

```
Arguments : query_word, (period)

return : title_list, category_list, body_list
```

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def body_extractor(link):
    
    soup = BeautifulSoup(requests.get(link).content, 'html.parser')
    category = soup.select_one(".category").text

    return category, soup.select_one("div.text").text.strip().replace("\n", "").replace("\r", "").replace("■", "").replace("▲", "")

def hankyerae_crawler(query, period=None):
    
    # data containers 
    titles = []
    links = []
    categories = []
    dates = []
    bodies = []
    
    i = 0 
    nobody = 0
    
    while True:
        print("-" * 30)
        print('{} page is start'.format(i))
        
        url = "http://search.hani.co.kr/Search?command=query&keyword={}&media=news&sort=d&period=all&datefrom=2000.01.01&dateto=2019.06.05&pageseq={}".format(query, i)
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'html.parser')
        
        if len(soup.select(".search-none")) != 0:
            print('***** no result! *****')
            break

        for art, da in zip(soup.select("dt > a"), soup.select(".date > dl > dd")):
            
            title = art.text
            if len(title) == 0:
                nobody += 1
                continue
                
            link = art["href"]
            if len(link) == 0:
                nobody += 1
                continue
                    
            date = da.text
            if len(date) == 0:
                date = "unknown_date"
            
            try:
                category, body = body_extractor(link)
                if len(body) == 0:
                    print("no body")
                    nobody += 1
                    continue
                if len(category) == 0:
                    category = 'unknown_category'
            
            except Exception as ex:
                print(ex)
                nobody += 1
                continue
                
            titles.append(title)
            links.append(link)
            categories.append(category) 
            dates.append(date)
            bodies.append(body)
            
        print('{} page is done'.format(i+1))
        i += 1

    return [titles, links, categories, dates, bodies], nobody

In [3]:
data_dust, nobody_dust = hankyerae_crawler("미세먼지", period=None)

------------------------------
0 page is start
0 page is done


In [4]:
data_dust

[['[ESC] 달리기를 느껴라! 러닝 기술 ABC',
  '“누진제 개편안, 원전·석탄발전 더 많이 가동하란 정책”',
  '‘세계 환경의 날’…창원서 전국 첫 수소버스 운행',
  "[날씨] 남부지방 낮기온 33도↑…전국 미세먼지 '보통'∼'나쁨'",
  '한강 품은 호텔 연계 오피스텔 ‘리버뷰 나루 하우스’ 눈길',
  "울산시-기업-NGO 함께 ‘미세먼지 저감 도시 숲' 조성",
  '235개 120만t…“헬프미” 전국은 쓰레기로 신음 중',
  '[날씨] 대구 낮 최고 34도 ‘무더위’…경기 일부 출근길 빗방울',
  '[날씨] 더위 속 경상내륙 폭염주의보…미세먼지 ‘보통’∼‘나쁨’',
  '[날씨] 전국 맑고 미세먼지 ‘보통’…제주도 오전 빗방울'],
 ['http://www.hani.co.kr/arti/specialsection/esc_section/896827.html',
  'http://www.hani.co.kr/arti/economy/marketing/896813.html',
  'http://www.hani.co.kr/arti/society/environment/896740.html',
  'http://www.hani.co.kr/arti/society/environment/896678.html',
  'http://www.hani.co.kr/arti/economy/property/896632.html',
  'http://www.hani.co.kr/arti/area/yeongnam/896582.html',
  'http://www.hani.co.kr/arti/society/society_general/896578.html',
  'http://www.hani.co.kr/arti/society/environment/896522.html',
  'http://www.hani.co.kr/arti/society/society_general/896353.html',
  'http://www.hani.co.kr/arti/society/environ